In [177]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [178]:
url = "https://www.koreabaseball.com/Player/Trade.aspx"
driver = webdriver.Chrome()
driver.get(url)

In [179]:
selYear_options = driver.find_elements(By.CSS_SELECTOR, "#selYear > option") # 2017 ~ 2025 옵션 elements
selYear_2020s = list(filter(lambda x : int(x.text) >= 2020, selYear_options)) # 2020 ~ 2025 옵션들만 필터링

In [180]:
search_button = driver.find_element(By.CSS_SELECTOR, "#btnSearch")

In [181]:
selYear_2020s[0].click()
search_button.click()
table = driver.find_element(By.CSS_SELECTOR, "#tblTradeList") # 현재 페이지의 표 추출하기
thead = table.find_elements(By.CSS_SELECTOR, "#tblTradeList > thead > tr > th") # colnames 추출하기

In [ ]:
player_move_list = []

for selYear in selYear_2020s:
    selYear.click() # 연도 옵션 선택 후
    search_button.click() # 검색 버튼 클릭하여 연도별 선수이동목록 불러오기
    time.sleep(1)
    while True:
        table = driver.find_element(By.CSS_SELECTOR, "#tblTradeList") # 현재 페이지의 표 추출하기
        tbody = table.find_elements(By.CSS_SELECTOR, "#tblTradeList > tbody > tr") # table의 각 row 추출하기
        for index, value in enumerate(tbody):
            td = value.find_elements(By.CSS_SELECTOR, "td")
            if len(td) > 1:
                player_move_list.append([x.text for x in td])
        
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "#contents > div.sub-content > div.paging > a.pg_next")
        except:
            break
        else:
            if "disabled" in next_button.get_attribute("class"): # 해당 년도의 마지막 페이지까지 이동한 경우
                break # 다음 년도로 이동
            else:
                next_button.click()
                time.sleep(1)

다음으로 넘어가는 버튼 업뜸


In [185]:
df = pd.DataFrame(player_move_list, columns=[x.text for x in thead])

In [201]:
df["포지션"] = df["선수"].apply(lambda x : x[x.find("(")+1:x.find(")")])
df["선수"] = df["선수"].apply(lambda x : x[:x.find("(")])

In [214]:
df.to_csv("player_moves/move_list_2020s.csv", index=False)